In [74]:
import sys
from pathlib import Path

# e.g. notebooks/analysis.ipynb → ../module_folder
module_path = Path.cwd().parent / "/module-fuzz/rewiring"
sys.path.insert(0, str(module_path))

import generate_verilog as gv  # now works!
import IO_map
import numpy as np

In [75]:
io_map = IO_map.create_IO_map("/module-fuzz/exp/curr_run/wrapper_modules")
max_input = len(io_map["input"])
max_output = len(io_map["output"])
gv.generate_verilog("/module-fuzz/exp/curr_run/wrapper_modules", "/module-fuzz/exp/curr_run", 5)

In [76]:
for input_size in range(1, max_input + 1, 2):
    print(input_size)
    #gv.generate_verilog("/module-fuzz/coverage_library_IO_flattened", "/module-fuzz/exp/curr_run", input_size, output_size=max_output, output_internal_wires=True)
    

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
221
223
225
227
229
231
233
235
237
239
241
243
245
247
249
251
253
255
257
259
261
263
265
267
269
271
273
275
277
279
281
283
285
287
289
291
293
295
297
299
301
303
305
307
309
311
313
315
317
319
321
323
325
327
329
331
333
335
337
339
341
343
345
347
349
351
353
355
357
359
361
363
365
367
369
371
373
375
377
379
381


In [77]:
print(max_input)
print(max_output)

382
477


In [ ]:
#!/usr/bin/env python3
import argparse, textwrap, subprocess, os, re, sys

TB_NAME      = "tb"
TB_SV_FILE   = f"{TB_NAME}.sv"
MAIN_CPP     = "sim_main.cpp"
OBJ_DIR      = "obj_dir"
WRAPPER_EXEC = os.path.join(OBJ_DIR, f"V{TB_NAME}")

TB_TEMPLATE = textwrap.dedent("""\
    `timescale 1ns/1ps
    `default_nettype none

    module {tb_name};
      parameter int IN_WIDTH  = {in_width};
      parameter int OUT_WIDTH = {out_width};
      parameter int CYCLES     = {cycles};

      // DUT interface
      logic clk = 0;
      logic rst_n;
      logic [IN_WIDTH-1:0]  in_flat;
      wire  [OUT_WIDTH-1:0] out_flat;

      // instantiate DUT
      top dut (
        .in_flat(in_flat),
        .out_flat(out_flat)
      );

      // counters
      integer i;
      integer z_count, x_count, normal_count;

      // clock
      always #5 clk = ~clk;

      initial begin
        $dumpfile("tb.vcd");
        $dumpvars(0, {tb_name});

        rst_n = 0;
        #10;
        rst_n = 1;

        z_count      = 0;
        x_count      = 0;
        normal_count = 0;

        for (i = 0; i < CYCLES; i++) begin
          @(posedge clk);
          in_flat = $urandom;

          #1; // wait for outputs

          // count bit-states
          for (int j = 0; j < OUT_WIDTH; j++) begin
            casez (out_flat[j])
              1'bz: z_count++;
              1'bx: x_count++;
              default: normal_count++;
            endcase
          end
        end

        $display("TOTAL_Z_BITS:      %0d", z_count);
        $display("TOTAL_X_BITS:      %0d", x_count);
        $display("TOTAL_NORMAL_BITS: %0d", normal_count);
        $finish;
      end
    endmodule
    """)

MAIN_CPP_TEMPLATE = textwrap.dedent("""\
    #include "V{tb_name}.h"
    #include "verilated.h"
    int main(int argc, char **argv) {{
      Verilated::commandArgs(argc, argv);
      V{tb_name}* tb = new V{tb_name};
      while (!Verilated::gotFinish()) {{
        tb->eval();
      }}
      delete tb;
      return 0;
    }}
    """)

def generate_tb(args):
    with open(TB_SV_FILE, "w") as f:
        f.write(TB_TEMPLATE.format(
            tb_name   = TB_NAME,
            in_width  = args.in_width,
            out_width = args.out_width,
            cycles    = args.cycles
        ))
    with open(MAIN_CPP, "w") as f:
        f.write(MAIN_CPP_TEMPLATE.format(tb_name=TB_NAME))
    print(f"Generated {TB_SV_FILE} + {MAIN_CPP}")

def run_verilator(args):
    # ensure output directory exists
    if os.path.isdir(OBJ_DIR):
        subprocess.run(["rm", "-rf", OBJ_DIR], check=True)
    cmd = [
        "verilator",
        "--cc",
        *args.dut,
        TB_SV_FILE,
        "-I /module-fuzz/exp/curr_run/wrapper_modules",
        "-I /module-fuzz/exp/curr_run/base_modules"
        "--exe", MAIN_CPP,
        "--build",
        "--Mdir", OBJ_DIR
    ]
    print("Invoking Verilator:", " ".join(cmd))
    subprocess.run(cmd, check=True)

def run_simulation():
    print("Running simulation...")
    p = subprocess.run([WRAPPER_EXEC], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.returncode != 0:
        print("Simulation failed:", p.stderr, file=sys.stderr)
        sys.exit(1)
    return p.stdout

def parse_and_report(output):
    z = int(re.search(r"TOTAL_Z_BITS:\s+(\d+)",      output).group(1))
    x = int(re.search(r"TOTAL_X_BITS:\s+(\d+)",      output).group(1))
    n = int(re.search(r"TOTAL_NORMAL_BITS:\s+(\d+)", output).group(1))
    print(output.strip())
    ratio = (z+x) / n if n else float('inf')
    print(f"\nRatio (Z+X) / NORMAL = {z + x} / {n} = {ratio:.3f}")

def main():
    p = argparse.ArgumentParser(
        description="Generate, build & run a generic SV TB that counts Z/X vs normal bits"
    )
    p.add_argument("--dut", nargs="+", required=True,
                   help="Verilog source(s) for your DUT (e.g., top.sv)")
    p.add_argument("--in_width",  type=int, required=True,
                   help="width of input bus in_flat")
    p.add_argument("--out_width", type=int, required=True,
                   help="width of output bus out_flat")
    p.add_argument("--cycles",    type=int, default=1000,
                   help="how many clock cycles to run")
    p.add_argument("--no-run", action="store_true",
                   help="only generate TB (don't compile/run)")
    args = p.parse_args()

    generate_tb(args)

    if args.no_run:
        print("Done. You can now manually invoke Verilator or run the sim.")
        return

    run_verilator(args)
    sim_out = run_simulation()
    parse_and_report(sim_out)

if __name__ == "__main__":
    main()
